In [115]:
# here we are using text for recognizing sentiments
# basically buillding classifier to classify whether text is srcastic or not
# for which we are using a dataset..from Rishabh Misra

# the dataset is in json format
# we need to convert it into python object format

In [141]:
import json

sentences = []
labels = []
urls = []


for data in open("Sarcasm_Headlines_Dataset.json",'r'):
    sentences.append(json.loads(data)['headline'])
    labels.append(json.loads(data)['is_sarcastic'])
    urls.append(json.loads(data)['article_link'])
    
print(sentences[10])
print(labels[10]) #labels = 1 if sarcastic otherwise 0
print(urls[10])

airline passengers tackle man who rushes cockpit in bomb threat
0
https://www.huffingtonpost.com/entry/airline-passengers-tackle-man-who-rushes-cockpit-in-bomb-threat_us_59302e57e4b07572bdbf9460


In [142]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

padded = pad_sequences(sequences,padding='post')

In [143]:
#print(word_index)
print(padded[0])
print(padded.shape)

[  308 15115   679  3337  2298    48   382  2576 15116     6  2577  8434
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
(26709, 40)


In [144]:
# for trainig a model we need to have training set and the testing set
# but in this case, we have just 26709 set of data and no partition for trainig and testing
# so before training we need to partition the dataset into trainig set and testing set ny slicing

In [145]:
training_size = int(0.8 * len(padded))
#testing_size = len(padded) - training_size

training_sentences = sentences[:training_size]
training_labels = labels[:training_size]

testing_sentences = sentences[training_size:]
testing_labels = labels[training_size:]

In [146]:
# we need to ensure that neural network only sees the training data and not the testing data
# that means we nedd to make sure the we tokenize only the trainig data for word_index

#earlier we used to pass sentences to toeknizer now we need to pass training_sentences to tokenizer

In [147]:
vocab_size=10000
embedding_dim = 16
max_length=100

In [148]:
tokenizer = Tokenizer(num_words=vocab_size,oov_token = '<OOV>')
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_sequences_padded = pad_sequences(training_sequences,padding='post',maxlen=100,truncating='post')

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_sequences_padded = pad_sequences(testing_sequences,padding='post',maxlen=100,truncating='post')

In [149]:
# now we tave toeknized the word using numeric tokens and formed sequences
# but how do we form the meaning of sentences just from number
# that means, how can we find whether the sentence is sarcastic just by looking at numbers

# here where the concepts of 'Embeddings' comes


In [150]:
import numpy as np
training_sequences_padded = np.array(training_sequences_padded)
training_labels = np.array(training_labels)
testing_sequences_padded = np.array(testing_sequences_padded)
testing_labels = np.array(testing_labels)

In [151]:
# building model
from keras.models import Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(24,activation='relu'),
    Dense(1,activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [152]:
#training model
history = model.fit(
    training_sequences_padded,
    training_labels,
    epochs=30,
    validation_data=(testing_sequences_padded,testing_labels),
    verbose=2,
)

Train on 21367 samples, validate on 5342 samples
Epoch 1/30
 - 2s - loss: 0.6672 - accuracy: 0.5907 - val_loss: 0.5908 - val_accuracy: 0.7703
Epoch 2/30
 - 2s - loss: 0.4555 - accuracy: 0.8255 - val_loss: 0.4014 - val_accuracy: 0.8220
Epoch 3/30
 - 2s - loss: 0.3276 - accuracy: 0.8703 - val_loss: 0.3558 - val_accuracy: 0.8506
Epoch 4/30
 - 2s - loss: 0.2764 - accuracy: 0.8926 - val_loss: 0.3452 - val_accuracy: 0.8527
Epoch 5/30
 - 2s - loss: 0.2406 - accuracy: 0.9070 - val_loss: 0.3395 - val_accuracy: 0.8555
Epoch 6/30
 - 2s - loss: 0.2138 - accuracy: 0.9186 - val_loss: 0.3455 - val_accuracy: 0.8504
Epoch 7/30
 - 2s - loss: 0.1924 - accuracy: 0.9282 - val_loss: 0.3502 - val_accuracy: 0.8549
Epoch 8/30
 - 2s - loss: 0.1736 - accuracy: 0.9348 - val_loss: 0.3688 - val_accuracy: 0.8472
Epoch 9/30
 - 2s - loss: 0.1596 - accuracy: 0.9409 - val_loss: 0.3740 - val_accuracy: 0.8523
Epoch 10/30
 - 2s - loss: 0.1450 - accuracy: 0.9478 - val_loss: 0.3905 - val_accuracy: 0.8469
Epoch 11/30
 - 2s - 

In [154]:
#using to establish sentiments for new sentences
input_sentence = ["That's just what I needed today!",'the weather today is bright and sunny']
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(input_sentence)
word_index = tokenizer.word_index

sequence = tokenizer.texts_to_sequences(input_sentence)

padded = pad_sequences(sequence,padding='post',maxlen=max_length,truncating='post')

predictions = model.predict(padded)
print(predictions)

[[0.02933377]
 [0.00061305]]
